In [1]:
from lightkurve import KeplerLightCurveFile, LightCurve, KeplerTargetPixelFile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import astropy.units as u
from astropy.stats import LombScargle
from astropy.convolution import Box2DKernel, convolve
from copy import deepcopy

In [ ]:
ID = 211416749

In [ ]:
tpf = KeplerTargetPixelFile.from_archive(ID, campaign=5, cadence='short', quality_bitmask='hardest')
aper = np.nanmedian(tpf.flux, axis=0) > np.nanpercentile(np.nanmedian(tpf.flux, axis=0), 70)
tpf.plot(aperture_mask=aper)
lc1 = tpf.to_lightcurve().normalize().remove_nans().remove_outliers().correct(windows=20).remove_outliers()


/Users/JZ/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:907: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/Users/JZ/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  
 15%|█▌        | 3/20 [00:02<00:16,  1.01it/s]

In [ ]:
lc1 = lc1.fill_nans()

In [ ]:
# The first ~3800 cadences have a large gap, it breaks our SFF detrender. Let's get rid of them.
tpf = KeplerTargetPixelFile.from_archive(ID, campaign=16, cadence='short', quality_bitmask='hardest')
aper = np.nanmedian(tpf.flux, axis=0) > np.nanpercentile(np.nanmedian(tpf.flux, axis=0), 70)
tpf.plot(aperture_mask=aper)
lc2 = tpf.to_lightcurve('all')[3800:].normalize().remove_nans().remove_outliers().correct(windows=20).remove_outliers()
lc2 = lc2.fill_nans()

In [ ]:
%reload_ext autoreload
%autoreload 2

from lightkurve.periodogram import Periodogram

pd1 = Periodogram.from_lightcurve(lc1)
pd2 = Periodogram.from_lightcurve(lc2)

In [ ]:
ax = pd1.plot()
ax.set_xlim(100, 400)
ax.set_ylim(0, 60)
pd2.plot(ax=ax)

In [ ]:
ax3  = lc1.periodogram().plot()
ax3.set_xlim(100, 400)
ax3.set_ylim(0, 60)

In [ ]:
pd1.normalized_plot()